In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from IPython.display import display
from csv import writer
import csv
import time
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

In [4]:
def get_jobs(num_jobs, verbose):
    
    s = Service('D:\Selenium driver\chromedriver2.exe')
    driver = webdriver.Chrome(service=s)
    driver.maximize_window()   
    url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword=business analyst&jobType=all&fromAge=-1&minSalary=0&includeNoSalaryJobs=true&radius=100&cityId=-1&minRating=0.0&industryId=-1&sgocId=-1&seniorityType=all&companyId=-1&employerSizes=0&applicationType=0&remoteWorkType=0'
    driver.get(url)
    location = WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.XPATH, '//*[@id="sc.location"]')))
    location.click()
    location.send_keys('Canada')
    search_button = driver.find_element_by_xpath('//*[@id="scBar"]/div/button').click()
    time.sleep(2)

    jobs = []
    while len(jobs) < num_jobs:
        
        time.sleep(4)
        
        job_buttons = driver.find_elements_by_xpath('//li[@data-ad-order-id="1136043"]')
       
        for job_button in job_buttons:
            
            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break

            job_button.click() 
            time.sleep(1)
            try:
                driver.find_element_by_xpath('//span[@alt="Close"]').click()
            except NoSuchElementException:
                pass  
            time.sleep(1)
            collected_successfully = False

            while not collected_successfully:
                try:
                    company_name = driver.find_element_by_xpath('.//div[@class="css-xuk5ye e1tk4kwz5"]').text
                    location = driver.find_element_by_xpath('.//div[@class="css-56kyx5 e1tk4kwz1"]').text
                    job_title = driver.find_element_by_xpath('.//div[@class="css-1j389vi e1tk4kwz2"]').text

                    collected_successfully = True
                except:
                    time.sleep(5)

                try:
                    job_description = driver.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').get_attribute('textContent')
                except NoSuchElementException:
                    job_description = -1


            try:
                salary_estimate = job_button.find_element_by_xpath('.//div[@class="css-1buaf54 pr-xxsm"]/span').text
            except NoSuchElementException:
                salary_estimate = -1 #You need to set a "not found value. It's important."

            try:
                rating = driver.find_element_by_xpath('.//span[@data-test="detailRating"]').text
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."

            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Job Description: {}".format(job_description))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))
            time.sleep(2)

            try:
                driver.find_element_by_xpath('.//div[@class="css-1ap6ha9 ef7s0la0"]').click()

                try:
                    headquarters = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Headquarters"]//following-sibling::*').text
                except NoSuchElementException:
                    headquarters = -1

                try:
                    size = driver.find_element_by_xpath('.//div[@class="d-flex justify-content-start css-daag8o e1pvx6aw2"]//span[text()="Size"]//following-sibling::*').text
                except NoSuchElementException:
                    size = -1

                try:
                    founded = driver.find_element_by_xpath('.//div[@class="d-flex justify-content-start css-daag8o e1pvx6aw2"]//span[text()="Founded"]//following-sibling::*').text
                except NoSuchElementException:
                    founded = -1

                try:
                    type_of_ownership = driver.find_element_by_xpath('.//div[@class="d-flex justify-content-start css-daag8o e1pvx6aw2"]//span[text()="Type"]//following-sibling::*').text
                except NoSuchElementException:
                    type_of_ownership = -1

                try:
                    industry = driver.find_element_by_xpath('.//div[@class="d-flex justify-content-start css-daag8o e1pvx6aw2"]//span[text()="Industry"]//following-sibling::*').text
                except NoSuchElementException:
                    industry = -1

                try:
                    sector = driver.find_element_by_xpath('.//div[@class="d-flex justify-content-start css-daag8o e1pvx6aw2"]//span[text()="Sector"]//following-sibling::*').text
                except NoSuchElementException:
                    sector = -1

                try:
                    revenue = driver.find_element_by_xpath('.//div[@class="d-flex justify-content-start css-daag8o e1pvx6aw2"]//span[text()="Revenue"]//following-sibling::*').text
                except NoSuchElementException:
                    revenue = -1

                try:
                    competitors = driver.find_element_by_xpath('.//div[@class="d-flex justify-content-start css-daag8o e1pvx6aw2"]//span[text()="Competitors"]//following-sibling::*').text
                except NoSuchElementException:
                    competitors = -1

            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                headquarters = -1
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1
                competitors = -1

            if verbose:
                print("Headquarters: {}".format(headquarters))
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("Competitors: {}".format(competitors))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
            
            #add job to jobs
            jobs.append({"Job Title" : job_title,
            "Salary Estimate" : salary_estimate,
            "Job Description" : job_description,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location,
            "Headquarters" : headquarters,
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue,
            "Competitors" : competitors})
        #Clicking on the "next page" button
        try:
            driver.find_element_by_xpath('//button[@aria-label="Next"]').click()
            time.sleep(3)
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            pass

    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.


In [5]:
df = get_jobs(570, False)
df.to_csv('BA Salary.csv', index = False)

Progress: 0/570
Progress: 1/570
Progress: 2/570
Progress: 3/570
Progress: 4/570
Progress: 5/570
Progress: 6/570
Progress: 7/570
Progress: 8/570
Progress: 9/570
Progress: 10/570
Progress: 11/570
Progress: 12/570
Progress: 13/570
Progress: 14/570
Progress: 15/570
Progress: 16/570
Progress: 17/570
Progress: 18/570
Progress: 19/570
Progress: 20/570
Progress: 21/570
Progress: 22/570
Progress: 23/570
Progress: 24/570
Progress: 25/570
Progress: 26/570
Progress: 27/570
Progress: 28/570
Progress: 29/570
Progress: 30/570
Progress: 31/570
Progress: 32/570
Progress: 33/570
Progress: 34/570
Progress: 35/570
Progress: 36/570
Progress: 37/570
Progress: 38/570
Progress: 39/570
Progress: 40/570
Progress: 41/570
Progress: 42/570
Progress: 43/570
Progress: 44/570
Progress: 45/570
Progress: 46/570
Progress: 47/570
Progress: 48/570
Progress: 49/570
Progress: 50/570
Progress: 51/570
Progress: 52/570
Progress: 53/570
Progress: 54/570
Progress: 55/570
Progress: 56/570
Progress: 57/570
Progress: 58/570
Progres

Progress: 462/570
Progress: 463/570
Progress: 464/570
Progress: 465/570
Progress: 466/570
Progress: 467/570
Progress: 468/570
Progress: 469/570
Progress: 470/570
Progress: 471/570
Progress: 472/570
Progress: 473/570
Progress: 474/570
Progress: 475/570
Progress: 476/570
Progress: 477/570
Progress: 478/570
Progress: 479/570
Progress: 480/570
Progress: 481/570
Progress: 482/570
Progress: 483/570
Progress: 484/570
Progress: 485/570
Progress: 486/570
Progress: 487/570
Progress: 488/570
Progress: 489/570
Progress: 490/570
Progress: 491/570
Progress: 492/570
Progress: 493/570
Progress: 494/570
Progress: 495/570
Progress: 496/570
Progress: 497/570
Progress: 498/570
Progress: 499/570
Progress: 500/570
Progress: 501/570
Progress: 502/570
Progress: 503/570
Progress: 504/570
Progress: 505/570
Progress: 506/570
Progress: 507/570
Progress: 508/570
Progress: 509/570
Progress: 510/570
Progress: 511/570
Progress: 512/570
Progress: 513/570
Progress: 514/570
Progress: 515/570
Progress: 516/570
Progress: 